In [1]:
import pandas as pd
import requests
from bs4 import BeautifulSoup
import datetime

In [2]:
#Esso Vissers Horst

#Texaco horst - https://bg.fuelo.net/gasstation/id/38090?lang=bg
#BP Express Sevenum - https://www.tankje.nl/GasStation/Details/2445
#TinQ Venray Henri Dunanstraat - https://www.tinq.nl/tankstations/venray-henri-dunantstraat#default

In [3]:
#BP Vissers Afferden
#Tango druten - https://www.brandstof-zoeker.nl/station/tango-druten-8931/
#TinQ druten - https://www.tinq.nl/tankstations/druten-industrieweg#default
#TinQ Winssen - Leegstraat - https://www.tinq.nl/tankstations/winssen-leegstraat#default
#TinQ Megen - Noord zuid - https://www.tinq.nl/tankstations/megen-noord-zuid#default

In [4]:
#Esso Vissers Bergh Noord
#shell babberich - https://bg.fuelo.net/gasstation/id/37998
#Kuster energy - https://www.tankje.nl/GasStation/Details/2587
#TinQ zevenaar - https://www.tinq.nl/tankstations/zevenaar-edisonstraat#default
#TinQ Millingen aan de Rijn - https://www.tinq.nl/tankstations/millingen-aan-de-rijn-heerbaan#default


In [5]:
urls = [['Texaco horst','https://bg.fuelo.net/gasstation/id/38090?lang=bg','Esso Vissers Horst'],
        ['BP Express Sevenum','https://www.tankje.nl/GasStation/Details/2445','Esso Vissers Horst'],
        ['TinQ Venray Henri Dunanstraat','https://www.tinq.nl/tankstations/venray-henri-dunantstraat#default','Esso Vissers Horst'],
        ['TinQ druten','https://www.tinq.nl/tankstations/druten-industrieweg#default','BP Vissers Afferden'],
        ['TinQ Winssen - Leegstraat','https://www.tinq.nl/tankstations/winssen-leegstraat#default','BP Vissers Afferden'],
        ['TinQ Megen - Noord zuid','https://www.tinq.nl/tankstations/megen-noord-zuid#default','BP Vissers Afferden'],
        ['shell babberich','https://bg.fuelo.net/gasstation/id/37998','Esso Vissers Bergh Noord'],
        ['Kuster energy','https://www.tankje.nl/GasStation/Details/2587','Esso Vissers Bergh Noord'],
        ['TinQ zevenaar','https://www.tinq.nl/tankstations/zevenaar-edisonstraat#default','Esso Vissers Bergh Noord'],
        ['TinQ Millingen aan de Rijn','https://www.tinq.nl/tankstations/millingen-aan-de-rijn-heerbaan#default','Esso Vissers Bergh Noord']
        ]

In [6]:
#Tinq
def Get_Tinq_Data(url):
    """
    Main function for getting data of fuel prices from TinQ.
    """
    page = requests.get(url=url)
    soup = BeautifulSoup(page.text,'html')
    fuel_names = Get_Fuel_Names_Tinq(soup=soup)
    fuel_prices = Get_Fuel_Prices_Tinq(soup=soup)
    return fuel_names,fuel_prices
    
def Get_Fuel_Names_Tinq(soup):
    """
    Get all fuel names that are at the Tinq stations.
    """
    benzine_soort = soup.find_all('div',class_='field field--name-taxonomy-term-title field--type-ds field--label-hidden field__item')
    benzine_namen = [benzine.text for benzine in benzine_soort]
    cleaned_benzine_namen = []
    
    for benzine in benzine_namen:
        benzine = benzine.replace('\n','').strip()
        cleaned_benzine_namen.append(benzine)
        
    return cleaned_benzine_namen

def Get_Fuel_Prices_Tinq(soup):
    """
    Get all prices of the fuel the station has.
    """
    prijzen = soup.find_all('div',class_='field field--name-field-prices-price-pump field--type-float field--label-hidden field__item')
    prijzen_lijst = [prijs.text for prijs in prijzen]
    cleaned_prijzen_namen = []
    for prijs in prijzen_lijst:
        prijs = prijs.replace('\n','').replace('/L','').replace('EUR','').replace('€ ','').strip()
        cleaned_prijzen_namen.append(prijs)
        
    return cleaned_prijzen_namen

In [7]:
#fuelo
def Get_Fuelo_Data(url):
    """
    Main function for getting data of fuel prices from Fuelo.
    """
    headers = {"Accept-Language": "en-US,en;q=0.5"}
    page = requests.get(url=url,headers=headers)
    soup = BeautifulSoup(page.text,'html')
    prices = Get_Fuel_Prices_Fuelo(soup=soup)
    fuel_names = Get_Fuel_Names_Fuelo(soup=soup) 
    return fuel_names,prices

def Get_Fuel_Prices_Fuelo(soup):
    """
    Get all prices of the fuel the station has.
    """
    prijzen = soup.find_all('span',itemprop='price')
    prices = []
    for x in prijzen:
        prices.append(x.text.strip())  
    return prices

def Get_Fuel_Names_Fuelo(soup):
    """
    Get all fuel names that are at the Tinq stations.
    """
    brandstoffen = soup.find_all('td',itemprop='name')
    brandstoffen_Cleaned = []
    for y in brandstoffen:
        brandstoffen_Cleaned.append(y.text.strip())
    return brandstoffen_Cleaned

In [8]:
#Tankje
def Get_Tankje_Data(url):
    """
    Main function for getting data of fuel prices from Tankje.
    """
    page = requests.get(url=url)
    soup = BeautifulSoup(page.text,'html')
    prices = Get_Fuel_Prices_Tankje(soup=soup)
    fuel_names = Get_Fuel_Names_Tankje(soup=soup) 
    return fuel_names,prices

def Get_Fuel_Prices_Tankje(soup):
    """
    Get all prices of the fuel the station has.
    """
    prijzen = soup.find_all('div',class_='h3 text-danger mb-2')
    prices = []
    for x in prijzen:
        prices.append(x.text.replace('€ ','').strip())  
    return prices

def Get_Fuel_Names_Tankje(soup):
    """
    Get all fuel names that are at the Tinq stations.
    """
    brandstoffen = soup.find_all('h4',class_='h6 mb-0')
    brandstoffen_Cleaned = []
    for y in brandstoffen:
        brandstoffen_Cleaned.append(y.text.strip())
    return brandstoffen_Cleaned

In [9]:
def Safe_To_File(row):
    for x in range(len(row[4])):
        df_new = pd.DataFrame([[row[0],row[1],row[2],row[3][x],row[4][x],row[5]]])
        df_new.to_csv("tank_data.csv", mode="a", header=False, index=False)

    

In [10]:
def get_Data():
    for url in urls:
        new_row = []
        if 'fuelo' in url[1]:
            fuel,price = Get_Fuelo_Data(url[1])
        elif 'tankje' in url[1]:
            fuel,price = Get_Tankje_Data(url[1])
        elif 'tinq' in url[1]:
            fuel,price = Get_Tinq_Data(url[1])
        new_row.append(url[0])
        new_row.append(url[1])
        new_row.append(url[2])
        new_row.append(fuel)
        new_row.append(price)
        new_row.append(datetime.datetime.today().strftime('%Y-%m-%d'))
        Safe_To_File(new_row)
        
get_Data()

In [11]:
df = pd.read_csv('tank_data.csv')
df

,Naam,URL,koppelbedrijf,brandstof,prijs,datum
0,Texaco horst,https://bg.fuelo.net/gasstation/id/38090?lang=bg,Esso Vissers Horst,Euro 95,"2,104",2025-12-05
1,Texaco horst,https://bg.fuelo.net/gasstation/id/38090?lang=bg,Esso Vissers Horst,Diesel,"1,988",2025-12-05
2,Texaco horst,https://bg.fuelo.net/gasstation/id/38090?lang=bg,Esso Vissers Horst,LPG,"0,781",2025-12-05
3,Texaco horst,https://bg.fuelo.net/gasstation/id/38090?lang=bg,Esso Vissers Horst,Diesel Premium,"0,000",2025-12-05
4,BP Express Sevenum,https://www.tankje.nl/GasStation/Details/2445,Esso Vissers Horst,Euro 95 (E10),"1,97",2025-12-05
...,...,...,...,...,...,...
59,TinQ Millingen aan de Rijn,https://www.tinq.nl/tankstations/millingen-aan...,Esso Vissers Bergh Noord,Superplus 98 E5,2.139,2025-12-07
60,TinQ Millingen aan de Rijn,https://www.tinq.nl/tankstations/millingen-aan...,Esso Vissers Bergh Noord,Diesel B7,1.609,2025-12-07
61,TinQ Millingen aan de Rijn,https://www.tinq.nl/tankstations/millingen-aan...,Esso Vissers Bergh Noord,LPG,0.749,2025-12-07
62,TinQ Millingen aan de Rijn,https://www.tinq.nl/tankstations/millingen-aan...,Esso Vissers Bergh Noord,Euro95 E10,1.849,2025-12-07
